# Parser for youtbue chanel vdud
https://www.youtube.com/channel/UCMCgOm8GZkHp8zJ6l7_hIuA

In [1]:
import pandas as pd
from selenium.webdriver import Chrome
from selenium.common.exceptions import NoSuchElementException 
import time
import datetime

import googleapiclient.discovery
import requests
import pickle

In [3]:
start = time.time()
# Creating dataframe, for the parsed data
cols = ['full_title', 'guest', 'length', 'views', 'likes', 'dislikes', 'date', 'comments_num']
data = []
df = pd.DataFrame(data = data, columns = cols)

In [4]:
# Link on the page for all the videos
all_Videos = 'https://www.youtube.com/c/vdud/videos'

In [5]:
#Starting browser
driver = Chrome('C:/WebDriver/chromedriver')
driver.get(all_Videos)
driver.maximize_window()

In [6]:
# Function to load all the videos
def scrollDown():
    driver.execute_script("window.scrollTo(0, 1660)")
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, 3200)")
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, 5000)")
    time.sleep(2)

In [7]:
scrollDown()

In [8]:
# Collect all the videos and get the list of links
play_list = driver.find_elements_by_id('thumbnail')

hrefs = [video.get_attribute('href') for video in play_list]

In [9]:
# It appears that some links were collected twice
hrefs = list(set(hrefs))
len(hrefs)

119

In [46]:
with open('hrefs.pickle', 'wb') as f:
    pickle.dump(hrefs, f)

# Functions declaration

In [10]:
# Check if the advertisment button exists
def check_if_btn_exists():
    try:
        driver.find_element_by_class_name('ytp-ad-skip-button-text')
    except NoSuchElementException:
        return False
    return True

In [11]:
# Getting length of the video in minutes
# If statement on the length of the string in order not to get length of the advertising video
def getLength():
    length = driver.find_element_by_class_name('ytp-time-duration').text
    
    if check_if_btn_exists() \
    and driver.find_element_by_class_name('ytp-ad-skip-button-text').text in ('Пропустить рекламу', 'Пропустить'):
        driver.find_element_by_class_name('ytp-ad-skip-button-text').click()
    
    if len(length)<5:
        time.sleep(1.5)
        return getLength()
    elif len(length.split(':')) == 3:
        return int(length.split(':')[0])*60 + int(length.split(':')[1])
    else:
        return int(length.split(':')[0])

In [12]:
# Getting full title and the name of the guest
def getTitle():
    full_title = driver.title
    
    title_end = full_title.find(' / ')
    if title_end < 0:
        title_end = full_title.find('.')
        if title_end < 0:
            title_end = full_title.find('(') - 1
            
    full_title = full_title[0:title_end]
    if full_title.find(' - ') > 0:
        guest = full_title[0:full_title.find('-')]
    elif full_title.find(' — ') > 0:
            guest = full_title[0:full_title.find('—')]
    elif full_title.find(' – ') > 0:
        guest = full_title[0:full_title.find('–')]
    elif full_title.find('. ') > 0:
            guest = full_title[0:full_title.find('.')]
    elif full_title.find(': ') > 0:
            guest = full_title[0:full_title.find(':')]
    else:
        guest = full_title
    return [full_title, guest]

In [13]:
# Getting statistics about the video
def getVideoStats():
    views = driver.find_element_by_class_name('yt-view-count-renderer').text
    views = int(views[0: views.find('п')-1].replace(' ', ''))
    likesDislikes = driver.find_element_by_id('top-level-buttons').text
    listLikDis = likesDislikes.replace(',', '.').split('\n')
    
    if listLikDis[0].find('ТЫС.') > 0:
        likes = float(listLikDis[0].split()[0])*1000
    elif listLikDis[0].find('МЛН') > 0:
        likes = float(listLikDis[0].split()[0])*1000000
        
    if listLikDis[1].find('ТЫС.') > 0:
        dislikes = float(listLikDis[1].split()[0])*1000
    elif listLikDis[1].find('МЛН') > 0:
        dislikes = float(listLikDis[1].split()[0])*1000000 
        
    return [views, likes, dislikes]

In [14]:
# Getting date of the interview
def getDate():
    date = driver.find_element_by_id('date').text.split()
    day = int(date[0][1:])
    
    months = ['янв', 'фев', 'мар', 'апр', 'мая', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек']
    for i, mon in enumerate(months):
        if mon == date[1][:3]:
            month = i+1
            
    year = int(date[2])
    return datetime.datetime(year, month, day)

In [15]:
# Getting the amount of comments
def getCommsNum():
    driver.execute_script("window.scrollTo(0, 880)")
    time.sleep(3)
    coms = driver.find_element_by_class_name('count-text').text
    return int(coms.split()[0])*1000 + int(coms.split()[1])

In [16]:
# Collecting all the information and updating the dataframe
def dfAppend():
    global df
    item = [getTitle()[0], getTitle()[1], getLength(), getVideoStats()[0], getVideoStats()[1], 
          getVideoStats()[2], getDate(), getCommsNum()]
    series = pd.Series(item, index=df.columns)
    df = df.append(series, ignore_index=True)

In [17]:
# Going through the collected links
def goHrefs():
    for href in hrefs:
        if type(href) == str:
            driver.get(href)
            time.sleep(2)
            dfAppend()

# Collecting information

In [18]:
t = time.time()
goHrefs()
print('Time to get all the data: {} mins'.format(round((time.time() - t) / 60, 2)))

Time to get all the data: 13.63 mins


In [19]:
df.shape

(118, 8)

In [20]:
df.head()

,full_title,guest,length,views,likes,dislikes,date,comments_num
0,Максим Фадеев - о конфликте с Эрнстом и русско...,Максим Фадеев,52,8621967,167000.0,11000.0,2017-05-16,30245
1,Невзоров - о Фараоне и ориентации Милонова,Невзоров,85,14349677,326000.0,26000.0,2018-01-30,47203
2,Колокольников,Колокольников,72,3884574,79000.0,9200.0,2018-05-22,11053
3,"Собчак - о Навальном, крестном и выборах",Собчак,96,14875828,251000.0,67000.0,2017-10-24,52065
4,"Щербаков - спецназ, панк-рок, любовь",Щербаков,99,21323332,789000.0,45000.0,2020-01-28,125871


In [21]:
df['likes/dislikes'] = df['likes']/df['dislikes']
df['views/likes'] = df['views']/df['likes']
df['views/comments_num'] = df['views']/df['comments_num']

In [ ]:
# Saving dataframe
df.to_csv('vdud.csv', index=False)

# Get comments through YouTube API

## Functions declaration

In [2]:
# get video Ids from previously collected links
def getVideoIds(hrefs):
    video_ids = []
    for href in hrefs:
        if type(href) == str:
            video_ids.append(href.split('?v=')[1])
    return video_ids

In [3]:
# Your api key
my_api_key = 'api_key'

In [5]:
# Getting first 100 comments
def getComms(video_id):
    api_service_name = "youtube"
    api_version = "v3"
    DEVELOPER_KEY = my_api_key

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)

    request = youtube.commentThreads().list(
        part = "snippet",
        videoId = video_id,
        maxResults = 100
    )
    init_response = request.execute()
    all_comms = []
    for item in init_response['items']:
        all_comms.append(item['snippet']['topLevelComment']['snippet']['textOriginal'])
        
    getNextComms(init_response['nextPageToken'], video_id, all_comms)
    
    return all_comms

In [29]:
# Getting all other comments
def getNextComms(page_token, video_id, all_comms):
    api_service_name = "youtube"
    api_version = "v3"
    DEVELOPER_KEY = my_api_key

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)

    request = youtube.commentThreads().list(
        part = "snippet",
        videoId = video_id,
        maxResults = 100,
        pageToken = page_token
    )
    
    try:
        response = request.execute()

        print(len(all_comms), end="\r")

        if len(response['items'])<97:
            for item in response['items']:
                all_comms.append(item['snippet']['topLevelComment']['snippet']['textOriginal'])
            return
        else:
            for item in response['items']:
                all_comms.append(item['snippet']['topLevelComment']['snippet']['textOriginal'])
            page_token = response['nextPageToken']
            return getNextComms(page_token, video_id, all_comms)
    except Exception as ex:
        print(ex)
        return 

In [26]:
# Start collectin comments
def getComsFromVideos(array_of_videos, n):
    all_video_comms = []
    try:
        for i in range(n, n+1):
            all_video_comms.append(getComms(array_of_videos[i]))
            print(i)
            print(i)
            if i==n+1:
                return all_video_comms
    except Exception as ex:
        print(ex)
    return all_video_comms

## Collecting comments

In [8]:
# Loading video links
with open('hrefs.pickle', 'rb') as f:
    hrefs = pickle.load(f)
len(hrefs)

Because of the google's limit for getting information from youtube. You can get around 1 billion comments in one day.
So I had to collect comments through the whole week, twenty videos for one day.

In [10]:
t = time.time()
all_comms_101_119 = getComsFromVideos(getVideoIds(hrefs), 101)
print('Time to get all comments from 20 videos: {} mins'.format(round((time.time() - t) / 60, 2)))

10100
10200
10300
10400
10500
1060
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
list index out of range
Time to get all comments from 20 videos: 43.95 mins


In [38]:
# Checking if were collected descent amount of comments through the ratio of replies to all comms
coms = list(df.comments_num[101:])
nums_no_replies = [len(text) for text in all_comms_101_119]

for i in range(len(all_comms_101_119)):
    print(df.iloc[101+i].guest, i)
    print(coms[i]-nums_no_replies[i])
    print('ratio of replies to all comms', (coms[i] - nums_no_replies[i])/coms[i])
    print('--------------------------------------------------')

Батыгин  0
8383
ratio of replies to all comms 0.17506891654832513
--------------------------------------------------
Сергей Супонев  1
3374
ratio of replies to all comms 0.23232114576878055
--------------------------------------------------
Агутин  2
5123
ratio of replies to all comms 0.25001220047825873
--------------------------------------------------
Мартиросян  3
3568
ratio of replies to all comms 0.16808781269138362
--------------------------------------------------
Пошлая Молли  4
6876
ratio of replies to all comms 0.21863275039745628
--------------------------------------------------
Познер  5
9042
ratio of replies to all comms 0.22041293908295345
--------------------------------------------------
MTV  6
4832
ratio of replies to all comms 0.227163744064689
--------------------------------------------------
Венедиктов  7
6229
ratio of replies to all comms 0.17493751228690987
--------------------------------------------------
Шнур  8
3317
ratio of replies to all comms 0.179715013

In [37]:
# Saving the list of comments
with open('all_comms_101_119.pickle', 'wb') as f:
    pickle.dump(all_comms_101_119, f)

In [50]:
# Uniting all comments in one column
df = pd.read_csv('vdud.csv')

all_comms_0_20 = pd.read_pickle('all_comms_0_20.pickle')
all_comms_21_40 = pd.read_pickle('all_comms_21_40.pickle')
all_comms_41_60 = pd.read_pickle('all_comms_41_60.pickle')
all_comms_61_80 = pd.read_pickle('all_comms_61_80.pickle')
all_comms_81_100 = pd.read_pickle('all_comms_81_100.pickle')
all_comms_101_118 = pd.read_pickle('all_comms_101_119.pickle')

all_comms = all_comms_0_20+all_comms_21_40+all_comms_41_60+all_comms_61_80+all_comms_81_100+all_comms_101_118
df['comms_text'] = all_comms

In [61]:
# Let's take a look
df[['guest', 'comms_text']].head(10)

,guest,comms_text
0,Максим Фадеев,[Кое-что из внутренностей:\n\n0:37 Что Фадееву...
1,Невзоров,[00:38 Что и кому Невзоров преподает в итальян...
2,Колокольников,[0:23 Сниматься в настоящем крематории в гробу...
3,Собчак,[Кое-что из внутренностей:\n\n00:24 Еще 4 меся...
4,Щербаков,[Что происходит?\n\n1:02 Щербаков стал суперзв...
5,Ходорковский,[Кое-что из внутренностей:\n\n00:29 Какое собы...
6,Влади (Каста),[Кое-что из внутренностей:\n\n1:38 Как в Росто...
7,Илья Найшуллер,[Кое-что из внутренностей:\n\n03:30 Почему кли...
8,IC3PEAK,[Еще одна ветка для на конкурса.\nНу и кое-что...
9,MORGENSHTERN,[Варианты на конкурс - сюда. Кое-что из внутре...


In [62]:
df.to_csv('vdud_coms.csv', index=False)

In [31]:
# date of collecting all except for comments
datetime.datetime.today().date()

datetime.date(2020, 11, 6)

In [32]:
print('Time to get everything: {} mins'.format(round((time.time() - start) / 60, 2)))

Time to get everything: 48.8 mins
